In [ ]:
"""
==========================================================
Titel:    DNB-SRU-Analyse und Schlagwortmatching mit DDC
Autor:    [Isabell Sickert]
Datum:    [16.10.2025]
Version:  2.0
==========================================================

Beschreibung:
Dieses Skript dient der automatisierten Abfrage, Anreicherung und Analyse
von Titeldaten der Deutschen Nationalbibliothek (DNB) anhand der SRU-Schnittstelle.
Es kombiniert bibliografische Metadaten mit Schlagwort- und DDC-Informationen,
führt eine Volltextsuche in vorhandenen Inhaltsverzeichnissen durch und erstellt eine strukturierte,
gewichtete Ergebnistabelle in Excel-Form.

----------------------------------------------------------
Ablauf und Hauptfunktionen:
----------------------------------------------------------

1) Schlagworte und DDC-Mapping einlesen:
   - Liest „idn.xlsx“ (Liste von DNB-IDNs) und
     „schlagworte_ddc_gewichtung.xlsx“ (Schlagwort-Mapping).
   - Erstellt daraus:
       • `keywords` → Liste aller Schlagwörter zur Textsuche
       • `weights` → Schlagwort-Gewichtungen (Relevanz)
       • `systematics` → Zuordnung Schlagwort → Systematik

2) SRU-Abfrage (query_dnb):
   - Für jede IDN wird die DNB-SRU-Schnittstelle aufgerufen.
   - Extrahiert bibliografische Informationen:
       • Titel, Zusatztitel, Autor, Verlag, Jahr
       • DDC-Codes (Fachklassifikation)
       • Sachgruppen (alternative Klassifikation)
       • GND-Schlagwörter (Personen- und Geoschlagwörter)
   - Erkennt, ob der Titel im DBSM-Bestand vorhanden ist.
   - Kürzt DDC-Codes automatisch bis zur ersten Null nach dem Punkt.
   - Schreibt alle Ergebnisse zurück in das Haupt-DataFrame.

3) DDC-Kürzungsfunktion:
   - Funktion `shorten_ddc_cell` säubert und kürzt DDC-Werte in Spalten.
   - Wird auf „DDC1“ und „DDC2“ angewendet, um saubere Klassifikationen
     für spätere Auswertungen zu erhalten.

4) Inhaltsverzeichnisse durchsuchen (find_keywords):
   - Ruft den Volltext eines DNB-Titels ab (`/04/text`).
   - Durchsucht den Text nach vordefinierten Schlagwörtern.
   - Zählt Treffer, berechnet die Gesamtgewichtung und bestimmt
     die am besten passende Systematik (höchste Relevanz).
   - Gibt zusätzlich Links zum Volltext und PDF zurück.

5) Schlagwort- und DDC-Mapping (apply_mapping):
   - Durchsucht zusätzlich Titel und Zusatztitel nach bekannten Begriffen.
   - Erhöht ggf. die Gesamtgewichtung und ergänzt Systematiken.
   - Dient der Feinjustierung des Klassifikationsergebnisses.

6) Filtern (aktuell deaktiviert):
   - Optional können Titel nach Gewichtung, DDC-Blacklist,
     Verlagen oder Inhalt (z. B. „Roman“) herausgefiltert werden.
   - Diese Filter sind vorbereitet, aber derzeit auskommentiert.

7) Export & Formatierung:
   - Speichert die Resultate als „neu_test2.xlsx“.
   - Erstellt klickbare Hyperlinks für Text- und PDF-URLs.
   - Ausgabe: Eine Excel-Tabelle mit allen relevanten
     Metadaten, Klassifikationen und Schlagworttreffern.

----------------------------------------------------------
Abhängigkeiten:
- pandas
- requests
- BeautifulSoup4
- openpyxl
- xml.etree.ElementTree
- re

----------------------------------------------------------
Ausgabe:
→ Datei: „neu_test2.xlsx“
Enthält alle analysierten DNB-Titel mit
  - bibliografischen Daten,
  - DDC- und Schlagwortzuordnungen,
  - Relevanzgewichtung,
  - Hyperlinks zu Text/PDF,
  - DBSM-Kennzeichnung.

==========================================================
"""


In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from openpyxl import load_workbook
import re

# ===============================
# 1) Schlagworte + DDC-Mapping einlesen
# ===============================
idn_df = pd.read_excel("idn.xlsx", dtype=str)

mapping_df = pd.read_excel("schlagworte_ddc_gewichtung.xlsx", dtype=str)
keywords = mapping_df[mapping_df["Typ"] == "Schlagwort"]["Begriff"].dropna().tolist()
weights = mapping_df.set_index("Begriff")["Gewichtung"].astype(int).to_dict()
systematics = mapping_df.set_index("Begriff")["Systematik"].to_dict()

# ===============================
# 2) SRU-Abfrage zuerst
# ===============================
def query_dnb(idn):
    url = f"https://services.dnb.de/sru/dnb?version=1.1&operation=searchRetrieve&query=idn={idn}&recordSchema=MARC21plus-xml"
    response = requests.get(url)
    if response.status_code != 200:
        return None, None, None, None, None, None, None, None, None, None

    root = ET.fromstring(response.text)
    ns = {'marc': 'http://www.loc.gov/MARC21/slim'}

    gnds1, gnds2 = [], []

    title, subtitle, author, publisher, year = None, None, None, None, None
    ddc_list, sachgruppe_list = [], []
    subjects, geo_subjects = [], []

    for record in root.findall(".//marc:record", ns):
        for datafield in record.findall("marc:datafield", ns):
            tag = datafield.get("tag")
            ind1 = datafield.get("ind1")

            if tag == "245":
                sub_a = datafield.find("marc:subfield[@code='a']", ns)
                sub_b = datafield.find("marc:subfield[@code='b']", ns)
                title = sub_a.text if sub_a is not None else None
                subtitle = sub_b.text if sub_b is not None else None

            elif tag == "100":
                sub_a = datafield.find("marc:subfield[@code='a']", ns)
                author = sub_a.text if sub_a is not None else None

            elif tag in ["260", "264"]:
                sub_b = datafield.find("marc:subfield[@code='b']", ns)
                sub_c = datafield.find("marc:subfield[@code='c']", ns)
                publisher = sub_b.text if sub_b is not None else None
                year = sub_c.text if sub_c is not None else None

            elif tag == "082":
                sub_a = datafield.find("marc:subfield[@code='a']", ns)
                if sub_a is not None:
                    if ind1 == "0":  # echte DDC
                        ddc_list.append(sub_a.text)
                    elif ind1 == "7":  # Sachgruppe
                        sachgruppe_list.append(sub_a.text)

            elif tag == "083":
                sub_a = datafield.find("marc:subfield[@code='a']", ns)
                if sub_a is not None:
                    if ind1 == "0":  # echte DDC
                        ddc_list.append(sub_a.text)
                    elif ind1 == "7":  # Sachgruppe
                        sachgruppe_list.append(sub_a.text)

            elif tag == "650":  # GNDS1
                for sub in datafield.findall("marc:subfield[@code='a']", ns):
                    if sub is not None:
                        gnds1.append(sub.text)

            elif tag == "651":  # GNDS2
                for sub in datafield.findall("marc:subfield[@code='a']", ns):
                    if sub is not None:
                        gnds2.append(sub.text)

    # DBSM prüfen
    dbsm_flag = None
    dbsm_standort = None

    for holding in root.findall(".//marc:record[@type='Holdings']", ns):
        for datafield in holding.findall("marc:datafield[@tag='852']", ns):
            sub_b = datafield.find("marc:subfield[@code='b']", ns)  # Standort
            if sub_b is not None:
                standort = sub_b.text
                if "dbsm" in standort.lower():
                    dbsm_flag = "x"
                    dbsm_standort = standort  # Standort merken
                    break

    # DDC nur bis zur ersten Null nach dem Punkt kürzen
    def shorten_ddc(ddc):
        if not ddc:
            return ""
        parts = ddc.split(".")
        if len(parts) == 2:
            # alles nach der ersten Null nach dem Punkt abschneiden
            match = re.match(r"(\d+\.\d*?)0", ddc)
            if match:
                return match.group(1)
            else:
                return ddc
        return ddc

    ddc_cleaned = "; ".join(shorten_ddc(d) for d in ddc_list if d)
    ddc1 = ddc_cleaned.split(";")[0] if ddc_cleaned else ""
    ddc2 = ddc_cleaned.split(";")[1] if len(ddc_cleaned.split(";")) > 1 else ""

    return (
        title or "",
        subtitle or "",
        author or "",
        publisher or "",
        year or "",
        dbsm_flag or "",
        dbsm_standort or "",
        ddc1,
        ddc2,
        "; ".join(sachgruppe_list) if sachgruppe_list else "",
        "; ".join(subjects) if subjects else "",
        "; ".join(geo_subjects) if geo_subjects else ""
    )



for idx, row in idn_df.iterrows():
    title, subtitle, author, publisher, year, dbsm_flag, dbsm_standort, ddc1, ddc2, sachgruppe, gnds1, gnds2 = query_dnb(row.iloc[0])
    idn_df.at[idx, "Titel"] = title
    idn_df.at[idx, "Zusatztitel"] = subtitle
    idn_df.at[idx, "Autor"] = author
    idn_df.at[idx, "Verlag"] = publisher
    idn_df.at[idx, "Jahr"] = year
    idn_df.at[idx, "DBSM-Bestand"] = dbsm_flag
    idn_df.at[idx, "DBSM-Standort"] = dbsm_standort
    idn_df.at[idx, "DDC1"] = ddc1
    idn_df.at[idx, "DDC2"] = ddc2
    idn_df.at[idx, "Sachgruppe"] = sachgruppe
    idn_df.at[idx, "GNDS1"] = gnds1
    idn_df.at[idx, "GNDS2"] = gnds2


# ===============================
# DDC auf erste Nachkommastelle kürzen
# ===============================
import re

def shorten_ddc_cell(cell_value):
    """Kürzt alle DDC-Codes in einer Zelle nach der ersten Null nach dem Punkt."""
    if pd.isna(cell_value) or not cell_value.strip():
        return ""
    
    parts = [part.strip() for part in cell_value.split(";")]
    shortened_parts = []
    for ddc in parts:
        match = re.match(r"(\d+\.\d*?)0", ddc)
        if match:
            shortened_parts.append(match.group(1))
        else:
            shortened_parts.append(ddc)
    return "; ".join(shortened_parts)

# Beide Spalten anwenden
for col in ["DDC1", "DDC2"]:
    if col in idn_df.columns:
        idn_df[col] = idn_df[col].apply(shorten_ddc_cell)

 

# ===============================
# 3) Inhaltsverzeichnisse durchsuchen
# ===============================
def find_keywords(idn):
    url_text = f"https://d-nb.info/{idn}/04/text"
    url_pdf = f"https://d-nb.info/{idn}/04/pdf"
    formatted = ""  # <-- Initialisierung
    total_weight = 0
    best_syst = None

    try:
        response = requests.get(url_text, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            text = soup.get_text().lower()

            word_counts = {}
            for word in keywords:
                pattern = re.compile(rf"\b\w*{re.escape(word.lower())}\w*\b", re.IGNORECASE)
                matches = pattern.findall(text)
                count = len(matches)
                if count > 0:
                    word_counts[word] = (count, weights.get(word, 1), systematics.get(word, ""))

            if word_counts:
                total_weight = sum(v[1] for v in word_counts.values())
                if len(word_counts) >= 3 or total_weight >= 3:
                    best_word, (best_count, best_weight, best_syst) = max(
                        word_counts.items(),
                        key=lambda x: (x[1][1], x[1][0])
                    )
                    formatted = "; ".join(
                        f"{w} (Gewicht={wt}, Treffer={ct})"
                        for w, (ct, wt, _) in word_counts.items()
                    )

    except requests.RequestException:
        formatted = "Fehler"

    return url_text or "", url_pdf or "", formatted or "", total_weight or 0, best_syst or ""





idn_df[["URL", "PDF-URL", "Gefundene Schlagwörter", "Gesamtgewichtung", "Gewinner-Systematik"]] = \
    idn_df.iloc[:, 0].apply(find_keywords).apply(pd.Series)

idn_df.rename(columns={idn_df.columns[0]: "IDN"}, inplace=True)
#idn_df = idn_df.dropna(subset=["Gefundene Schlagwörter"])

# ===============================
# 4) Schlagwort- und DDC-Mapping
# ===============================
def apply_mapping(row):
    total_weight = row["Gesamtgewichtung"]
    systematik = row["Gewinner-Systematik"]
    found_extra = []

    # Titel + Zusatztitel durchsuchen
    for text in [row["Titel"], row["Zusatztitel"]]:
        if pd.notna(text):
            t = text.lower()
            for w in keywords:
                if re.search(rf"\b{re.escape(w.lower())}\b", t):
                    total_weight += weights.get(w, 1)
                    found_extra.append(w)
                    if not systematik:
                        systematik = systematics.get(w)

    # DDC prüfen
    #if row["DDC"]:
        #for ddc_code in row["DDC"].split(";"):
            #ddc_code = ddc_code.strip()
            #if not ddc_code:
             #   continue
            #if ddc_code in systematics:
             #   systematik = systematics[ddc_code]
              #  found_extra.append(f"DDC {ddc_code}")
            #for key in systematics.keys():
             #   if key.startswith(f"{ddc_code} +"):
              #      systematik = systematics[key]
               #     found_extra.append(key)

    return pd.Series([total_weight, systematik, "; ".join(found_extra)])


idn_df[["Gesamtgewichtung", "Gewinner-Systematik", "Zusätzliche Treffer"]] = \
    idn_df.apply(apply_mapping, axis=1)

# ===============================
# 5) Filtern
# ===============================
#idn_df = idn_df[idn_df["Gesamtgewichtung"] >= 4]
#idn_df = idn_df[~idn_df["Zusatztitel"].str.contains(r"\broman\b", case=False, na=False)]

#blacklist_ddc = ["360", "590", "910", "333", "355", "370", "510", "610", "621", "690", "796", "797", "798", "799", "914"]
#idn_df = idn_df[~idn_df["DDC"].str.startswith(tuple(blacklist_ddc), na=False)]

#verlage_df = pd.read_excel("Verlage.xlsx")
#blacklist = verlage_df["Verlag"].dropna().unique().tolist()
#idn_df = idn_df[~idn_df["Verlag"].isin(blacklist)]

# ===============================
# 6) Excel speichern + Hyperlinks
# ===============================
output_file = "neu_test2.xlsx"
idn_df.to_excel(output_file, index=False)

wb = load_workbook(output_file)
ws = wb.active
col_url = [c[0] for c in enumerate(ws[1]) if c[1].value == "URL"][0] + 1
col_pdf = [c[0] for c in enumerate(ws[1]) if c[1].value == "PDF-URL"][0] + 1

for row in range(2, ws.max_row + 1):
    if ws.cell(row=row, column=col_url).value:
        ws.cell(row=row, column=col_url).hyperlink = ws.cell(row=row, column=col_url).value
        ws.cell(row=row, column=col_url).style = "Hyperlink"
    if ws.cell(row=row, column=col_pdf).value:
        ws.cell(row=row, column=col_pdf).hyperlink = ws.cell(row=row, column=col_pdf).value
        ws.cell(row=row, column=col_pdf).style = "Hyperlink"

wb.save(output_file)
print("Fertig! Ergebnisse in neu_test2.xlsx")


C:\Users\sickerti\AppData\Local\Temp\ipykernel_17296\3338783985.py:188: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(response.text, "html.parser")


Fertig! Ergebnisse in neu_test2.xlsx
